
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y1", label="ph")

In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-22 02:03:46.060502: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-22 02:03:46.102959: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 02:03:46.103001: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 02:03:46.104064: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-22 02:03:46.109792: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-22 02:03:46.110320: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [9]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [10]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [11]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [12]:
tester.Loop(sort_by='r2',
            boundarie=0.5,
            hidden_sizes = [[15, 7], [30, 15], [30, 15, 7]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02, 0.1])

Testando combinacao1: Hidden Size=[15, 7], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  1.310999601999356
Stopped at epoch:  17
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  2.5803450429993973
Stopped at epoch:  62
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.8534213399998407
Stopped at epoch:  20
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.6760772620000353
Stopped at epoch:  19
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.2533014489999914
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.33872381699984544
Stopped at epoch:  8
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 7 ++++++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_3_1_0,0.528782,0.477975,0.403854,0.545087,0.088154,0.083425,0.053892,0.069527
model_3_1_1,0.536197,0.488346,0.414622,0.553825,0.086767,0.081767,0.052918,0.068191
model_3_1_2,0.578541,0.495318,0.426636,0.561029,0.078846,0.080653,0.051832,0.067090
model_3_1_18,0.607898,0.251907,0.293124,0.389120,0.073353,0.119553,0.063902,0.093364
model_3_1_11,0.612242,0.320256,0.351408,0.443179,0.072541,0.108630,0.058633,0.085102
model_3_1_3,0.613149,0.425519,0.515955,0.547251,0.072371,0.091808,0.043758,0.069196
model_3_1_12,0.613450,0.326823,0.349254,0.446216,0.072315,0.107580,0.058827,0.084638
model_3_1_9,0.614154,0.329475,0.366428,0.452465,0.072183,0.107157,0.057275,0.083683
model_3_1_19,0.614219,0.257823,0.320918,0.400131,0.072171,0.118607,0.061389,0.091681
model_3_1_10,0.614300,0.328668,0.364989,0.451618,0.072156,0.107285,0.057405,0.083812


DataFrame salvo em ../results/metrics_3_1
+++++++++++ [3_2] | 1 ++++++++++++++++++
Elapsed time:  0.3261327230011375
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [3_2] | 2 ++++++++++++++++++
Elapsed time:  0.28375293699900794
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [3_2] | 3 ++++++++++++++++++
Elapsed time:  0.2827130550012953
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [3_2] | 4 ++++++++++++++++++
Elapsed time:  0.29189317199961806
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [3_2] | 5 ++++++++++++++++++
Elapsed time:  0.2863003820002632
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [3_2] | 6 ++++++++++++++++++
Elapsed time:  0.3270156799990218
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [3_2] | 7 ++++++++++++++++++
Elapsed time:  0.3059413009

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_3_2_1,0.626759,0.536792,0.493611,0.528269,0.069825,0.073833,0.048094,0.061720
model_3_2_0,0.628756,0.516745,0.616553,0.557336,0.069451,0.077029,0.036417,0.057917
model_3_2_2,0.629270,0.550895,0.452794,0.523422,0.069355,0.071585,0.051970,0.062355
model_3_2_3,0.631758,0.557135,0.447685,0.525702,0.068890,0.070590,0.052455,0.062056
model_3_2_4,0.635532,0.563371,0.438322,0.526525,0.068184,0.069597,0.053345,0.061949
model_3_2_5,0.635970,0.564245,0.439428,0.527468,0.068102,0.069457,0.053239,0.061825
model_3_2_6,0.638208,0.567094,0.446702,0.531789,0.067683,0.069003,0.052549,0.061260
model_3_2_7,0.640166,0.565207,0.457543,0.534275,0.067317,0.069304,0.051519,0.060935
model_3_2_8,0.641476,0.569325,0.456683,0.536637,0.067072,0.068647,0.051601,0.060625
model_3_2_10,0.642497,0.572633,0.444969,0.534769,0.066881,0.068120,0.052713,0.060870


DataFrame salvo em ../results/metrics_3_2
+++++++++++ [3_3] | 1 ++++++++++++++++++
Elapsed time:  0.331748697999501
Stopped at epoch:  9
1/1 [==============================] - 0s 18ms/step
+++++++++++ [3_3] | 2 ++++++++++++++++++
Elapsed time:  0.42079155200008245
Stopped at epoch:  9
1/1 [==============================] - 0s 21ms/step
+++++++++++ [3_3] | 3 ++++++++++++++++++
Elapsed time:  0.605120234999049
Stopped at epoch:  12
1/1 [==============================] - 0s 15ms/step
+++++++++++ [3_3] | 4 ++++++++++++++++++
Elapsed time:  0.37693982500059064
Stopped at epoch:  7
1/1 [==============================] - 0s 15ms/step
+++++++++++ [3_3] | 5 ++++++++++++++++++
Elapsed time:  0.3280552889991668
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [3_3] | 6 ++++++++++++++++++
Elapsed time:  0.34035115799997584
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [3_3] | 7 ++++++++++++++++++
Elapsed time:  0.3071511140

...

DataFrame salvo em ../results/metrics_5_9
Testando combinacao6: Hidden Size=[30, 15], regularizer=0.02, learning_rate=0.1
+++++++++++ [6_0] | 1 ++++++++++++++++++
Elapsed time:  3.370666019000055
Stopped at epoch:  87
1/1 [==============================] - 0s 14ms/step
+++++++++++ [6_0] | 2 ++++++++++++++++++
Elapsed time:  0.3919076700003643
Stopped at epoch:  10
1/1 [==============================] - 0s 16ms/step
+++++++++++ [6_0] | 3 ++++++++++++++++++
Elapsed time:  0.47018113100057235
Stopped at epoch:  9
1/1 [==============================] - 0s 26ms/step
+++++++++++ [6_0] | 4 ++++++++++++++++++
Elapsed time:  0.8852367910003522
Stopped at epoch:  19
1/1 [==============================] - 0s 15ms/step
+++++++++++ [6_0] | 5 ++++++++++++++++++
Elapsed time:  0.32283742599975085
Stopped at epoch:  7
1/1 [==============================] - 0s 21ms/step
+++++++++++ [6_0] | 6 ++++++++++++++++++
Elapsed time:  0.4083444629995938
Stopped at epoch:  8
1/1 [==============================] -

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_9_4_0,0.506399,0.346361,-0.271214,0.239896,0.092342,0.177167,0.084566,0.133590
model_9_4_1,0.517564,0.338217,-0.239371,0.238920,0.090253,0.179374,0.082447,0.133762
model_9_4_2,0.527614,0.329559,-0.217184,0.235802,0.088373,0.181721,0.080971,0.134309
model_9_4_3,0.536585,0.320229,-0.202941,0.230722,0.086694,0.184250,0.080024,0.135202
model_9_4_4,0.544588,0.310108,-0.194468,0.223969,0.085197,0.186993,0.079460,0.136389
model_9_4_5,0.551718,0.299129,-0.190405,0.215727,0.083864,0.189969,0.079190,0.137838
model_9_4_6,0.558065,0.287352,-0.190057,0.206176,0.082676,0.193161,0.079167,0.139516
model_9_4_7,0.563717,0.274898,-0.192660,0.195542,0.081619,0.196536,0.079340,0.141385
model_9_4_8,0.568731,0.261873,-0.198031,0.183952,0.080681,0.200067,0.079697,0.143422
model_9_4_9,0.573184,0.248426,-0.205693,0.171608,0.079848,0.203712,0.080207,0.145592


DataFrame salvo em ../results/metrics_9_4
+++++++++++ [9_5] | 1 ++++++++++++++++++
Elapsed time:  0.38679732700074965
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [9_5] | 2 ++++++++++++++++++
Elapsed time:  0.325785287999679
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [9_5] | 3 ++++++++++++++++++
Elapsed time:  0.33517025699984515
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [9_5] | 4 ++++++++++++++++++
Elapsed time:  0.3357143299999734
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [9_5] | 5 ++++++++++++++++++
Elapsed time:  0.4412665529998776
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [9_5] | 6 ++++++++++++++++++
Elapsed time:  0.37584661199980474
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [9_5] | 7 ++++++++++++++++++
Elapsed time:  0.3045643629

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_9_5_5,0.502862,0.219601,-0.101371,0.102880,0.093003,0.113986,0.284592,0.194272
model_9_5_4,0.525116,0.551965,0.036327,0.298883,0.088840,0.065441,0.249011,0.151827
model_9_5_3,0.566295,0.651527,0.467654,0.576638,0.081136,0.050899,0.137557,0.091679
model_9_5_2,0.575654,0.667552,0.530355,0.617568,0.079386,0.048558,0.121356,0.082816
model_9_5_1,0.588801,0.683923,0.618060,0.672663,0.076926,0.046167,0.098693,0.070885
model_9_5_0,0.591092,0.703374,0.657130,0.701548,0.076498,0.043326,0.088597,0.064630


DataFrame salvo em ../results/metrics_9_5
+++++++++++ [9_6] | 1 ++++++++++++++++++
Elapsed time:  0.458791516999554
Stopped at epoch:  8
1/1 [==============================] - 0s 16ms/step
+++++++++++ [9_6] | 2 ++++++++++++++++++
Elapsed time:  0.30340419599997404
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [9_6] | 3 ++++++++++++++++++
Elapsed time:  0.30987337500118883
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [9_6] | 4 ++++++++++++++++++
Elapsed time:  0.3415628549992107
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [9_6] | 5 ++++++++++++++++++
Elapsed time:  0.25327060100062226
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [9_6] | 6 ++++++++++++++++++
Elapsed time:  0.24785168900052668
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [9_6] | 7 ++++++++++++++++++
Elapsed time:  0.256932930